# The purpose of this exercise is to show performance by designated market area - commonly referred to as DMA
Overview
The sales_archive.db contains four tables: dmas, sales, transactions, visits. The purpose of this exercise is to show performance by designated market area 

Questions 
1. What’s the total sales amount by designated market area (DMA)? List the name (not the id) and total sales amount for each DMA.

2. Average Order Value (AOV) is defined as sales.amount divided by transactions.transaction_count. List the name (not the id) and average order value for each DMA on January 1st, 2019. Order the result set from highest average order value to lowest.

3. For each DMA, calculate the average, lowest, and highest sales.amount for the month of January 2019. List the name (not the id), average sales amount, minimum sales amount, and maximum sales amount for each DMA.

4. Seems like something may be wrong with the data in our visits table. For the month of February 2019, list the name (not the id) and the frequency (count of occurrences where the condition is true) where a DMA’s visit count is greater than the transaction count. Order by the DMA name.

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

con = create_engine("sqlite:///sales_archive.db")
con.table_names()

[]

In [2]:
# 1. What’s the total sales amount by designated market area (DMA)? 
pd.read_sql_query("SELECT SUM(amount) FROM sales" , con)

OperationalError: (sqlite3.OperationalError) no such table: sales
[SQL: SELECT SUM(amount) FROM sales]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
# 1. List the name (not the id) and total sales amount for each DMA. -->
pd.read_sql_query("select name,sum(amount) from dmas inner join sales on dmas.id = sales.dma_id group by name order by sum(amount)", con).head()

In [ ]:
# 2. Average Order Value (AOV) is defined as sales.amount divided by transactions.transaction_count.
# 2 List the name (not the id) and total sales amount for each DMA. --> List the name (not the id) and average order value for each DMA on January 1st, 2019. Order the result set from highest average order value to lowest.
query2 = '''select b.name, sum_amount/transaction_count as aov
from  (
	select date,dmas.id,name,sum(amount) sum_amount 
	from dmas 
	inner join sales 
		on dmas.id = sales.dma_id  
	group by date,dmas.id,name
) b
join transactions t 
	on b.id = t.dma_id and t.date = b.date
where t.date = '2019-01-01'
group by  b.name,sum_amount/transaction_count
order by sum_amount/transaction_count desc
'''
pd.read_sql_query(query2, con)

3. For each DMA, calculate the average, lowest, and highest sales.amount for the month of January 2019. List the name (not the id), average sales amount, minimum sales amount, and maximum sales amount for each DMA.

In [ ]:
query3 = '''select name, avg(amount),min(amount),max(amount)
from sales s
join dmas d
	on d.id = s.dma_id
where date < '2019-02-01' --month of January 2019. 
group by name
'''
pd.read_sql_query(query3, con)

4. Seems like something may be wrong with the data in our visits table. For the month of February 2019, list the name (not the id) and the frequency (count of occurrences where the condition is true) where a DMA’s visit count is greater than the transaction count. Order by the DMA name.y

In [ ]:
query4 = '''

select name, count(*) as frequency
from visits  v
join transactions t
	on v.dma_id = t.dma_id  AND v.date =t.date
join dmas d
	on d.id = t.dma_id
where v.date between  '2019-02-01' and '2019-03-01' and visit_count > transaction_count 
group by name 
order by name 
--where a DMA’s visit count is greater than the transaction count
'''

pd.read_sql_query(query3, con)